In [2]:
import numpy as np
import random
import cmath

In [3]:
#Rz Gate
def Rz(l):
  return np.matrix(
      [
          [np.exp(-l*1j/2), 0],
          [0, np.exp(l*1j/2)]
      ]
  )

In [4]:
#Rx Gate
def Rx(l):
  return np.matrix(
      [
          [np.cos(l / 2), -1j * np.sin(l / 2)],
          [-1j * np.sin(l / 2), np.cos(l / 2)]
      ]
  )

In [5]:
#Conversion of spherical angles to cartesian

def x(phi, theta):
  return np.sin(theta) * np.cos(phi)

def y(phi, theta):
  return np.sin(theta) * np.sin(phi)

def z(theta):
  return np.cos(theta)

In [6]:
#Functions determining the phase acculumated by Rx and Rz gates

#Eq. 100
def gamma_x(l, phi, theta):
  return -np.arctan(np.cos(phi) / (cot(l / 2) * cot(theta / 2) + np.sin(phi)))

#Eq. 103
def gamma_z(l):
  return -l / 2

In [7]:
def cot(x):
  return 1 / np.tan(x)

In [8]:
#Given an initial state and a final state, determine the unitary gate that maps the initial state to the final state
#Also determine the value of alpha for calculation of the matrix form of U
def U(gamma_i, phi_i, theta_i, gamma_f, phi_f, theta_f):
  x_i = x(phi_i, theta_i)
  y_i = y(phi_i, theta_i)
  z_i = z(theta_i)

  x_f = x(phi_f, theta_f)
  y_f = y(phi_f, theta_f)
  z_f = z(theta_f)

  lambda_1 = -np.arctan(z_i / y_i)
  lambda_2 = np.arccos(x_i)
  lambda_3 = np.arccos(x_f)
  lambda_4 = np.arctan(z_f / y_f)

  if phi_i < 0: 
    lambda_2 = -lambda_2
  if phi_f < 0:
    lambda_3 = -lambda_3
  
  #Eq. 118, 119
  lambda_p = 2 * (
                    gamma_i - gamma_f 
                  + gamma_x(lambda_1, phi_i, theta_i)
                  + gamma_z(lambda_3 - lambda_2)
                  + gamma_x(lambda_4, lambda_3, np.pi / 2)
                )
  
  #Eq. 132
  alpha = gamma_x(lambda_1, np.pi + phi_i, np.pi - theta_i) + gamma_z(lambda_3 - lambda_2) + gamma_x(lambda_4, lambda_3 - np.pi, np.pi / 2) + lambda_p / 2

  return Rx(lambda_4) @ Rz(lambda_3) @ Rx(lambda_p) @ Rz(-lambda_2) @ Rx(lambda_1), alpha

In [22]:
i = 0
cases = 10000
for i in range(cases):
  gamma_i = random.uniform(0, 2 * np.pi)
  gamma_f = random.uniform(0, 2 * np.pi)

  phi_i = random.uniform(-np.pi, np.pi)
  phi_f = random.uniform(-np.pi, np.pi)

  theta_i = random.uniform(0, np.pi)
  theta_f = random.uniform(0, np.pi)

  psi_i = np.exp(gamma_i * 1j) * np.array([np.cos(theta_i / 2), np.exp(phi_i * 1j) * np.sin(theta_i / 2)])
  psi_f = np.exp(gamma_f * 1j) * np.array([np.cos(theta_f / 2), np.exp(phi_f * 1j) * np.sin(theta_f / 2)])

  U1, alpha = U(gamma_i, phi_i, theta_i, gamma_f, phi_f, theta_f)

  #Eq. 133
  matrix1 = np.matrix(
    [
        [np.cos(theta_f / 2) * np.cos(theta_i / 2), np.exp(-phi_i * 1j) * np.cos(theta_f / 2) * np.sin(theta_i / 2)],
        [np.exp(phi_f * 1j) * np.sin(theta_f / 2) * np.cos(theta_i / 2), np.exp((phi_f - phi_i) * 1j) * np.sin(theta_f / 2) * np.sin(theta_i / 2)]
    ]
  )

  matrix2 = np.matrix(
      [
          [np.sin(theta_f / 2) * np.sin(theta_i / 2), -np.exp(-phi_i * 1j) * np.sin(theta_f / 2) * np.cos(theta_i / 2)],
          [-np.exp(phi_f * 1j) * np.cos(theta_f / 2) * np.sin(theta_i / 2), np.exp((phi_f - phi_i) * 1j) * np.cos(theta_f / 2) * np.cos(theta_i / 2)]
      ]
  )

  U2 = np.exp((gamma_f - gamma_i) * 1j) * matrix1 + np.exp(alpha * 1j) * matrix2
  
  #Comparing |psi_f> to U|psi_i>
  psi_diff = U1.dot(psi_i) - psi_f
  if not np.all(np.abs(psi_diff) < 1e-10):
    print('Failure: U1')
    print(psi_i)
    print(psi_f)
    print(psi_diff)
    break

  #Comparing U obtained by the 5 rotation gates to U obtained from Eq. 133
  U_diff = U2 - U1
  if not np.all(np.abs(U_diff) < 1e-10):
    print('Failure: U2')
    print(U1)
    print(U2)
    print(U_diff)
    break